In [ ]:
import iris
import numpy as np
import pandas as pd
import holoviews as hv
import geoviews as gv

import cartopy
from cartopy import crs as ccrs
from cartopy import feature as cf

from bokeh.tile_providers import STAMEN_TONER
from bokeh.models import WMTSTileSource

hv.notebook_extension('bokeh')

So far we have largely focused on the matplotlib backend since it has general support for projecting data to different geographic projections using cartopy. However the bokeh backend offers more advanced tools to interactively explore data, but is currently restricted to displaying data in web Mercator coordinates. Through integration with cartopy it is however still possible to project points, geometries and even images from arbitrary coordinate systems to web Mercator and render it in bokeh. In addition bokeh provides a general interface to render web based map tile sources.

# WMTS Tile Sources

The ``WMTS`` element accepts tile source API URLs when working with matplotlib. To use the ``WMTS`` Element with bokeh you must supply a ``WMTSTileSource`` object. Here we provide a list of common APIs, further open tile sources you could use can be found [here](http://wiki.openstreetmap.org/wiki/Tile_servers).

In [ ]:
tiles = {'OpenMap': WMTSTileSource(url='http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png'),
         'ESRI': WMTSTileSource(url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{X}/{Y}.jpg'),
         'Wikipedia': WMTSTileSource(url='https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'),
         'Stamen Toner': STAMEN_TONER}

Now we can lay these Elements out in an NdLayout by wrapping each ``WMTSTileSource`` in a ``WMTS`` Element and specifying both the extent and the coordinate reference system of these extents. Note that the extents are only required when displaying the tile source on its own, when it is overlaid with some data the data determines the extent.

In [ ]:
%%opts WMTS [width=450 height=250 xaxis=None yaxis=None]
hv.NdLayout({name: gv.WMTS(wmts, extents=(0, -90, 360, 90), crs=ccrs.PlateCarree())
            for name, wmts in tiles.items()}, kdims=['Source']).cols(2)

You may also supply a tuple of tile sources consisting of a bokeh WMTSTileSource object and a simple string URL, allowing the WMTS element to be used for both matplotlib and bokeh rendering without having to declare separate objects.

## Plotting data

One of the main benefits of plotting data with bokeh is the interactivity it allows. Here we will load a dataset of all the major cities in the world with their population counts over time:

In [ ]:
cities = pd.read_csv('./assets/cities.csv', encoding="ISO-8859-1")
population = hv.Dataset(cities, kdims=['City', 'Country', 'Year'])
cities.head()

Now we can convert this dataset to a set of points mapped by the latitude and longitude and containing the population, country and city as values. The longitudes and latitudes in the dataframe are supplied in simple Plate Carree coordinates, which we declare on the. The ``to`` conversion interface let's us do this in a succinct manner. Note that since we did not assign the Year dimension to the points key or value dimensions it is automatically assigned to a HoloMap, automatically rendering the data as an an animation:

In [ ]:
%%opts Overlay [width=700 height=400 xaxis=None yaxis=None] Points (size=0.005 cmap='viridis') [tools=['hover'] color_index=2]
(gv.WMTS(tiles['Wikipedia']) *\
population.to(gv.Points, kdims=['Longitude', 'Latitude'],
              vdims=['Population', 'City', 'Country'], crs=ccrs.PlateCarree()))

## Chloropleths

The tutorial on [Geometries](Geometries.ipynb) already covers how we can easily load a shapefile and combine it with a dataset to generate a chloropleths. Here we load the same dataset in the exact same way:

In [ ]:
shapefile = './assets/boundaries/boundaries.shp'
shapes = cartopy.io.shapereader.Reader(shapefile)
referendum = pd.read_csv('./assets/referendum.csv')
referendum = hv.Dataset(referendum)

The bokeh backend provides the additional ability to associate a hover label with each geometry in the plot. By activating the hover tool via the ``tools`` plot option HoloViews will automatically display the dimensions and values for any ``NdOverlay`` key dimensions, i.e. the index of cholorpleth and the value associated with each shape. This makes it trivial to generate a complex interactive plot:

In [ ]:
%%opts Shape (cmap='viridis') [xaxis=None yaxis=None tools=['hover'] width=400 height=500]
gv.Shape.from_records(shapes.records(), referendum, on='code', value='leaveVoteshare',
                     index='name', crs=ccrs.PlateCarree(), group='EU Referendum')

### Images

The bokeh backend also provides basic support for working with images. In this example we will simply load a very simple iris Cube and display it ovelaid by the coastlines feature. Note that the bokeh backend does not project the image to web mercator projection, instead relying on regridding the data. This means the actual display may be subtly different from the more powerful matplotlib backend.

In [ ]:
%%opts Overlay [width=600 height=400] Image (cmap='viridis')
surface_temp = iris.load_cube(iris.sample_data_path('air_temp.pp'))
hv.Dataset(surface_temp).to.image() * gv.Feature(cf.COASTLINE)